In [49]:
import pandas as pd
import numpy as np

# Read data

In [50]:
train = pd.read_csv('../Data/kaggle/train.csv')
test = pd.read_csv('../Data/kaggle/test.csv')

In [51]:
train.sample()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
608,609,1,2,"Laroche, Mrs. Joseph (Juliette Marie Louise La...",female,22.0,1,2,SC/Paris 2123,41.5792,NaN,C


# concat train and test
왜 합쳐줘야 하나요?

In [52]:
train_drop_Survived = train.drop('Survived', axis=1)

In [53]:
train_drop_Survived['tag'] = 'train'
test['tag'] = 'test'

In [54]:
df = pd.concat([train_drop_Survived, test])

In [55]:
# Age, Fare에 null 값이 있네요. Age에는 빈 값들이 많고, Fare는 딱 하나 비어있어요.
df.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
tag               0
dtype: int64

### Fare가 null 인 것을 index로 찾아볼까요?

In [56]:
df.loc[df['Fare'].isnull(), 'Fare']

152   NaN
Name: Fare, dtype: float64

### Q. 비어있는 Fare 데이터를 적당한 값을 채워넣어 주세요.

In [57]:
df.loc[df['Fare'].isnull(), 'Fare'] = df.groupby('Pclass')['Fare'].median()[3] 

### Q. 비어있는 Age 데이터를 적당한 값으로 채워넣어 주세요.

방법 1. 전체 데이터의 평균, 중앙, 최빈 중에 하나로 채워넣어준다.

In [58]:
df['Age_v1'] = df['Age'].fillna(np.mean(df['Age']))

방법 2-1. 다른 변수들을 1개를 활용해 그룹 별 평균, 중앙, 최빈값들로 넣어준다.   
예를 들어, Pclass별로 Age가 눈에 띄게 다른 것을 발견했다면 Pclass별 평균 또는 중앙값으로 null 값들을 대체해준다.

In [59]:
df['Age_v2'] = df.groupby(['Pclass'])['Age'].transform(lambda x: x.fillna(x.mean()))

방법 2-2. 다른 변수들을 2개 이상을 활용해 그룹 별 평균, 중앙, 최빈값들로 넣어준다.
예를 들어 티켓의 클래스, 승선 항구 별로 Age가 다르다는 것을 발견했다면 Pclass, Embarked의 조합 그룹 별 평균 또는 중앙값으로 null 값들을 대체해준다.

In [60]:
df['Age_v3'] = df.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.mean()))

방법 3. 회귀 분석으로 null 값들을 예측한다.

In [62]:
from sklearn.linear_model import LinearRegression

In [63]:
df['Sex'] = df['Sex'].replace('male', 0).replace('female', 1)

In [64]:
train_X_Age = df[~df['Age'].isnull()][['Pclass', 'Sex']]
train_y_Age = df[~df['Age'].isnull()]['Age']

test_X_Age = df[df['Age'].isnull()][['Pclass', 'Sex']]

print(train_X_Age.shape)
print(train_y_Age.shape)
print(test_X_Age.shape)

(1046, 2)
(1046,)
(263, 2)


In [66]:
model = LinearRegression()
model.fit(train_X_Age, train_y_Age)
prediction = model.predict(test_X_Age)

In [67]:
df['Age_v4'] = df['Age']
df.loc[df['Age'].isnull(), 'Age_v4'] = prediction

각 방법들로 만든 Age 값들의 비교

In [70]:
df[['Age_v1', 'Age_v2', 'Age_v3', 'Age_v4']].describe()

,Age_v1,Age_v2,Age_v3,Age_v4
count,1309.000000,1309.000000,1309.000000,1309.000000
mean,29.881138,29.348219,29.376186,29.298707
std,12.883193,13.127850,13.169015,13.178502
min,0.170000,0.170000,0.170000,0.170000
25%,22.000000,22.000000,22.000000,21.743801
50%,29.881138,26.000000,26.000000,26.000000
75%,35.000000,37.000000,37.000000,36.345401
max,80.000000,80.000000,80.000000,80.000000


### Q. Embarked 컬럼의 경우에는 어떻게 해주면 좋을까요?

In [44]:
# Answer

### 사용할 컬럼들에 null 값이 없는지 최종적으로 확인하세요.

In [72]:
df.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          2
tag               0
Age_v1            0
Age_v2            0
Age_v3            0
Age_v4            0
dtype: int64

# 작업물을 저장하세요.
* [Difference between .pickle and .csv](https://stackoverflow.com/questions/48770542/what-is-the-difference-between-save-a-pandas-dataframe-to-pickle-and-to-csv)

In [73]:
df.to_pickle('./df.p')